In [1]:
import pandas as pd
from bokeh.models import ColumnDataSource, Range1d
from bokeh.plotting import figure, show
from bokeh.client import push_session
from bokeh.io import curdoc
import random

In [2]:
def add_galton_board(p, n):
    '''
    '''
    color = '#C19A6B'
    
    lines_x = [[-(n+1), -0.5], [0.5, n+1]]
    lines_y = [[0, (n+1)-0.5], [(n+1)-0.5, 0]]
    p.multi_line(lines_x, lines_y, line_width=3.0, line_color=color)
    
    pegs_x = []
    pegs_y = []
    for row in xrange(n):
        for col in xrange(row+1):
            pegs_x.append(-row+col*2)
            pegs_y.append(n-row)
            #print -i+j*2, n-i
    p.circle(pegs_x, pegs_y, fill_color=color, line_color=None, radius=0.3)

    return

In [3]:
def add_galton_ball(p, n=20):
    '''
    '''

    p.line([0], [n], line_width=5.0, line_color='red', name='trail')
    p.circle([0], [n], fill_color='red', line_color=None, radius=0.3, name='ball')
    
    return

In [4]:
def add_distro(p, n=20):
    '''
    '''
    
    df = pd.DataFrame({'x': range(-n+1, n+1, 2), 
                       'y': [-n+1]*n,
                       'width': 1.5, 
                       'height': 2.,
                       'count': 0
                      })
    ds = ColumnDataSource(df)
    p.rect('x', 'y', width='width', height='height', source=ds, name='distro')

In [5]:
def get_galton_plot(n=20):
    '''
    '''
    
    p = figure(x_range=Range1d(-(n+2), n+2), y_range=Range1d(-n+1, n+1))
    add_galton_board(p, n)
    add_galton_ball(p)
    add_distro(p)
    
    return p

In [6]:
def update(n=20):
    ball = p.select_one({'name': 'ball'})
    trail = p.select_one({'name': 'trail'})
    distro = p.select_one({'name': 'distro'})

    sign = random.choice([-1, 1])
    ball_x = list(ball.data_source.data['x'])
    ball_y = list(ball.data_source.data['y'])
    distro_count = list(distro.data_source.data['count'])
                 
    #if ball_y[-1] > 1:
    #    x = ball_x[-1] + sign
    #    y = ball_y[-1] - 1
    #    ball_x.append(x)
    #    ball_y.append(y)
    #else:
    #    distro_count[(ball_x[-1] + n - 1)/2] += 1
    #    norm = 1.0*sum(distro_count)
    #    distro.data_source.data['height'] = list([2.0*n*count/norm for count in distro_count])
    #
    #    ball_x = [0]
    #    ball_y = [n]
    
    ball_x = [random.choice([-1,1]) if x > 0 else 0 for x in xrange(n)]
    ball_x = [sum(ball_x[:i+1]) for i in xrange(len(ball_x))]
    ball_y = range(n,0,-1)
    distro_count[(ball_x[-1] + n - 1)/2] += 1
    norm = 1.0*sum(distro_count)
    maxi = max(distro_count)
    factor = n * norm / maxi
    height = [factor*count/norm for count in distro_count]
    distro.data_source.data['height'] = height
    distro.data_source.data['y'] = [-n+0.5*x for x in height]
    
    ball.data_source.data = {'x': ball_x, 'y': ball_y}
    trail.data_source.data = {'x': list(ball_x), 'y': list(ball_y)}
    distro.data_source.data['count'] = distro_count

In [7]:
if __name__ == '__main__':
    # You must first run "bokeh serve" to view this example
    session = push_session(curdoc())
    p = get_galton_plot()
    curdoc().add_periodic_callback(update, 50)
    session.show(p)
    session.loop_until_closed()